In [1]:
!pip install -q -U google-generativeai python-dotenv

In [1]:
import google.generativeai as genai
from IPython.display import Markdown, Image
import os, json
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

/Users/pattssun/FitHub/fithub-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
actual_data = {
    "images/1.png": {
        "calories": 357.81,
        "protein": 20.00,
        "carb": 20.96,
        "fat": 21.69
    },
    "images/2.png": {
        "calories": 655.99,
        "protein": 93.24,
        "carb": 23.34,
        "fat": 20.22
    },
    "images/3.png": {
        "calories": 351.89,
        "protein": 12.10,
        "carb": 57.50,
        "fat": 9.77
    },
    "images/4.png": {
        "calories": 184.99,
        "protein": 24.54,
        "carb": 8.90,
        "fat": 6.14
    },
    "images/5.png": {
        "calories": 139.56,
        "protein": 5.40,
        "carb": 14.66,
        "fat": 6.64
    },
    "images/6.png": {
        "calories": 950.40,
        "protein": 74.77,
        "carb": 54.13,
        "fat": 45.30
    },
    "images/7.png": {
        "calories": 284.69,
        "protein": 13.84,
        "carb": 14.42,
        "fat": 18.80
    },
    "images/8.png": {
        "calories": 557.73,
        "protein": 25.77,
        "carb": 23.81,
        "fat": 42.54
    },
    "images/9.png": {
        "calories": 583.38,
        "protein": 42.29,
        "carb": 52.42,
        "fat": 23.33
    },
    "images/10.png": {
        "calories": 163.86,
        "protein": 2.51,
        "carb": 28.49,
        "fat": 5.30
    },
    "images/11.png": {
        "calories": 428.67,
        "protein": 24.47,
        "carb": 4.90,
        "fat": 35.14
    },
    "images/12.png": {
        "calories": 506.36,
        "protein": 77.39,
        "carb": 16.41,
        "fat": 13.48
    }
}

In [7]:
def get_nutrition_info(image_path):
    # Upload the file
    sample_file = genai.upload_file(path=image_path)

    # Run Gemini model
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    prompt = """
    You are an expert nutritionist with extensive experience in visual food analysis and portion estimation. Your task is to provide the most accurate nutrition information possible for the given food image.

    Instructions:
    1. Carefully analyze the image, considering that it was taken 50 cm above the food item.
    2. Identify all visible food items and ingredients in the image.
    3. For each identified food item and ingredient, get the per gram calories and macronutrients (protein, carbohydrates, fat) based on data from the USDA Food and Nutrient Database. 
    4. Estimate the weight of each food item and ingredient in grams, taking into account the known distance from the camera and visual cues.
    5. Calculate the final nutritional content by multiplying the per gram values from step 3 with the estimated weights from step 4.
    6. Sum up the total nutritional content for all items.
    4. Consider the preparation method (e.g., fried, grilled, raw) as it affects nutritional content.
    5. Factor in any visible condiments, sauces, or dressings.
    6. Use your expertise to estimate the most likely recipe or preparation method if not immediately apparent.
    7. Round your final estimates to the nearest whole number for simplicity.
    9. If you're uncertain about any aspect, provide your best estimate and indicate the level of confidence.

    Output your nutritional analysis in the following JSON format, ensuring all values are integers:
    {
        "calories": <calories>,
        "protein": <protein>,
        "carb": <carb>,
        "fat": <fat>
    }
    Do not output anything else.

    Remember, accuracy is crucial. Use all available visual cues and your nutritional expertise to provide the most precise estimates possible.
    """
    response = model.generate_content([sample_file, prompt])

    return response.text

In [8]:
def get_percent_error(image_path):    
    actual_values = actual_data[image_path]
    predicted_values = json.loads(get_nutrition_info(image_path))
    
    output = {}
    for key in actual_values:
        actual = actual_values[key]
        predicted = predicted_values[key]
        percent_error = abs((predicted - actual) / actual) * 100
        output[key] = {
            "actual": actual,
            "predicted": predicted,
            "percent_error": round(percent_error, 2)
        }
    
    return output

In [9]:
percent_error_1 = get_percent_error("images/1.png")
percent_error_1

I0000 00:00:1722477139.802587 6997231 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1722477139.838279 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722477142.447073 6997231 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


{'calories': {'actual': 357.81, 'predicted': 500, 'percent_error': 39.74},
 'protein': {'actual': 20.0, 'predicted': 28, 'percent_error': 40.0},
 'carb': {'actual': 20.96, 'predicted': 46, 'percent_error': 119.47},
 'fat': {'actual': 21.69, 'predicted': 28, 'percent_error': 29.09}}

In [10]:
percent_error_2 = get_percent_error("images/2.png")
percent_error_2

{'calories': {'actual': 655.99, 'predicted': 608, 'percent_error': 7.32},
 'protein': {'actual': 93.24, 'predicted': 63, 'percent_error': 32.43},
 'carb': {'actual': 23.34, 'predicted': 32, 'percent_error': 37.1},
 'fat': {'actual': 20.22, 'predicted': 33, 'percent_error': 63.2}}

In [11]:
percent_error_3 = get_percent_error("images/3.png")
percent_error_3

{'calories': {'actual': 351.89, 'predicted': 340, 'percent_error': 3.38},
 'protein': {'actual': 12.1, 'predicted': 8, 'percent_error': 33.88},
 'carb': {'actual': 57.5, 'predicted': 60, 'percent_error': 4.35},
 'fat': {'actual': 9.77, 'predicted': 10, 'percent_error': 2.35}}

In [13]:
percent_error_4 = get_percent_error("images/4.png")
percent_error_4

{'calories': {'actual': 184.99, 'predicted': 367, 'percent_error': 98.39},
 'protein': {'actual': 24.54, 'predicted': 33, 'percent_error': 34.47},
 'carb': {'actual': 8.9, 'predicted': 28, 'percent_error': 214.61},
 'fat': {'actual': 6.14, 'predicted': 18, 'percent_error': 193.16}}

In [14]:
percent_error_5 = get_percent_error("images/5.png")
percent_error_5

{'calories': {'actual': 139.56, 'predicted': 300, 'percent_error': 114.96},
 'protein': {'actual': 5.4, 'predicted': 8, 'percent_error': 48.15},
 'carb': {'actual': 14.66, 'predicted': 36, 'percent_error': 145.57},
 'fat': {'actual': 6.64, 'predicted': 14, 'percent_error': 110.84}}

In [15]:
percent_error_6 = get_percent_error("images/6.png")
percent_error_6

{'calories': {'actual': 950.4, 'predicted': 640, 'percent_error': 32.66},
 'protein': {'actual': 74.77, 'predicted': 35, 'percent_error': 53.19},
 'carb': {'actual': 54.13, 'predicted': 90, 'percent_error': 66.27},
 'fat': {'actual': 45.3, 'predicted': 20, 'percent_error': 55.85}}

In [16]:
percent_error_7 = get_percent_error("images/7.png")
percent_error_7

{'calories': {'actual': 284.69, 'predicted': 415, 'percent_error': 45.77},
 'protein': {'actual': 13.84, 'predicted': 20, 'percent_error': 44.51},
 'carb': {'actual': 14.42, 'predicted': 30, 'percent_error': 108.04},
 'fat': {'actual': 18.8, 'predicted': 25, 'percent_error': 32.98}}

In [17]:
percent_error_8 = get_percent_error("images/8.png")
percent_error_8


{'calories': {'actual': 557.73, 'predicted': 495, 'percent_error': 11.25},
 'protein': {'actual': 25.77, 'predicted': 18, 'percent_error': 30.15},
 'carb': {'actual': 23.81, 'predicted': 35, 'percent_error': 47.0},
 'fat': {'actual': 42.54, 'predicted': 34, 'percent_error': 20.08}}

In [18]:
percent_error_9 = get_percent_error("images/9.png")
percent_error_9

{'calories': {'actual': 583.38, 'predicted': 730, 'percent_error': 25.13},
 'protein': {'actual': 42.29, 'predicted': 50, 'percent_error': 18.23},
 'carb': {'actual': 52.42, 'predicted': 90, 'percent_error': 71.69},
 'fat': {'actual': 23.33, 'predicted': 35, 'percent_error': 50.02}}

In [19]:
percent_error_10 = get_percent_error("images/10.png")
percent_error_10

{'calories': {'actual': 163.86, 'predicted': 347, 'percent_error': 111.77},
 'protein': {'actual': 2.51, 'predicted': 11, 'percent_error': 338.25},
 'carb': {'actual': 28.49, 'predicted': 61, 'percent_error': 114.11},
 'fat': {'actual': 5.3, 'predicted': 8, 'percent_error': 50.94}}

In [23]:
percent_error_11 = get_percent_error("images/11.png")
percent_error_11

{'calories': {'actual': 428.67, 'predicted': 408, 'percent_error': 4.82},
 'protein': {'actual': 24.47, 'predicted': 36, 'percent_error': 47.12},
 'carb': {'actual': 4.9, 'predicted': 21, 'percent_error': 328.57},
 'fat': {'actual': 35.14, 'predicted': 21, 'percent_error': 40.24}}

In [24]:
percent_error_12 = get_percent_error("images/12.png")
percent_error_12

{'calories': {'actual': 506.36, 'predicted': 436, 'percent_error': 13.9},
 'protein': {'actual': 77.39, 'predicted': 39, 'percent_error': 49.61},
 'carb': {'actual': 16.41, 'predicted': 27, 'percent_error': 64.53},
 'fat': {'actual': 13.48, 'predicted': 24, 'percent_error': 78.04}}